# 2406 New feedback AB analysis

Changing user feedback from "like-dislike" to "like-next"

In [47]:
from datetime import datetime
import os

import pandas as pd
import polars as pl
import numpy as np
import psycopg2
from dotenv import load_dotenv

In [4]:
load_dotenv()
DATABASE_URL = os.environ['DATABASE_URL']
conn = psycopg2.connect(DATABASE_URL)

In [5]:
user_df = pl.read_database('select * from public.user', conn)

In [6]:
meme_df = pl.read_database("select id, meme_source_id, raw_meme_id, status, type, telegram_file_id, caption, language_code, created_at, updated_at, published_at, ocr_result #>> '{}' ocr_result from public.meme", conn)

In [7]:
res = []
dates = pd.date_range(start='2024-06-06', end='2024-06-09', freq='d').strftime('%Y-%m-%d')
for d_from, d_to in zip(dates, dates[1:]):
    res.append(pl.read_database(f"select * from public.user_meme_reaction where sent_at >= '{d_from}'::timestamp and sent_at < '{d_to}'::timestamp", conn))
res = [df for df in res if len(df)]

In [8]:
user_meme_df = pl.concat(res)

In [12]:
user_meme_df = user_meme_df.filter(pl.col('reacted_at') >= datetime(2024, 6, 6, 21))

In [22]:
user_meme_df = user_meme_df.with_columns(pl.when(pl.col('user_id') % 4 == 0).then(pl.lit('test')).otherwise(pl.lit('control')).alias('group'))

In [38]:
n_reactions_df = user_meme_df.group_by(['user_id', 'group']).len()

In [70]:
users_by_gr = {}
for gr in ['test', 'control']:
    users_by_gr[gr] = user_meme_df.filter(pl.col('group') == gr).select(pl.col('user_id').unique())

In [71]:
n_reactions = {'test': [], 'control': []}
for i in range(1000):
    for gr in ['test', 'control']:
        cur_users = users_by_gr[gr].sample(len(users_by_gr[gr]), with_replacement=True, shuffle=True, seed=i+42)
        cnt = n_reactions_df.filter(pl.col('group') == gr).join(cur_users, on='user_id').select(pl.sum('len')).to_numpy()[0, 0]
        if gr == 'control':
            cnt /= 3
        n_reactions[gr].append(cnt)

In [74]:
for gr in ['test', 'control']:
    mean = np.array(n_reactions[gr]).mean()
    std = np.array(n_reactions[gr]).std() / np.sqrt(len(n_reactions[gr]))
    print(f'{gr} {mean:.2f} +- {1.98 * std:.2f}')

test 17823.67 +- 174.76
control 17293.70 +- 94.92
